# Wildfire Susceptibility mapping

**Abstract**

Table of Contents : 

- [Preprocessing]()

## Libraries

In [46]:
import numpy as np 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 4)

## Exploratory Data Analysis

In [47]:
# Boundaries french departments
french_departments_boundaries_path = '../data/departments_boundaries/departements-20170102-simplified/departements-20170102.shp'

# Historical inventory of Forest Fire Burnt Area
historical_burnt_areas_path = '../data/historical_burnt_areas/effis_layer/modis.ba.poly.shp'

# Digital Elevation Model and derived topographic variable
eudem_path = '../data/eudem/euden/EUD_CP-DEMS_3500025000-AA.tif'
slope_path = '../data/eudem/slope/EUD_CP-SLOP_3500025000-AA.tif'
aspect_path = '../data/eudem/aspect/EUD_CP-ASPC_3500025000-AA.tif'
hillshare_path = '../data/eudem/hillshade/EUD_CP-HLSD_3500025000-AA.tif'

# Land Cover
land_cover_1990_path = '../data/corine_cover/land_cover_1990/u2000_clc1990_v2020_20u1_raster100m/DATA/U2000_CLC1990_V2020_20u1.tif'
land_cover_1990_path = '../data/corine_cover/land_cover_2018/u2000_clc1990_v2020_20u1_raster100m/DATA/U2000_CLC1990_V2020_20u1.tif'

# Protected area map
protected_areas_path = '../data/protected_areas/ens/ens.shp'

### Boundaries of French departments

In [48]:
dep_boundaries = gpd.read_file(french_departments_boundaries_path, encoding='utf-8')
dep_boundaries.head()

DriverError: ../data/departments_boundaries/departements-20170102-simplified/departements-20170102.shp: No such file or directory

In [ ]:
dep_boundaries['surf_ha'].hist(log=True, bins=100)

plt.title('Distribution of departments surface in France')
plt.show()

In [ ]:
dep_boundaries.sort_values('surf_ha', ascending=False)

In [ ]:
# keep departments of interest
pyrenees_boundaries = dep_boundaries[(dep_boundaries.code_insee == '64') | (dep_boundaries.code_insee == '65')]
pyrenees_boundaries.head()

In [ ]:
# display area of interst
pyrenees_boundaries.plot()

plt.title('Pyrenées')
plt.show()

### Historical inventory of Forest Fire Burnt Area

In [ ]:
burnt_areas = gpd.read_file(historical_burnt_areas_path)
burnt_areas.head()

In [ ]:
burnt_areas['FIREDATE'] = pd.to_datetime(burnt_areas['FIREDATE'])
burnt_areas.dtypes

In [ ]:
burnt_areas['FIREYEAR'] = burnt_areas['FIREDATE'].apply(lambda x: x.year)
burnt_areas.head()

In [ ]:
bins = burnt_areas['FIREYEAR'].max() - burnt_areas['FIREYEAR'].min() + 1

burnt_areas['FIREYEAR'].hist(bins=bins)

plt.title('Number of fires per year')
plt.xlabel('Year')
plt.ylabel('Number of fires')

plt.show()

In [ ]:
burnt_areas['CLASS'].value_counts().plot(kind='bar')

plt.title('Class of fires')

plt.show()

In [ ]:
burnt_areas_pyrenees = burnt_areas[(burnt_areas['COUNTRY'] == 'FR') & ((burnt_areas['PROVINCE'] == 'Pyrénées-Atlantiques') |(burnt_areas['PROVINCE'] == 'Hautes-Pyrénées'))]

In [ ]:
burnt_areas_pyrenees['PROVINCE'].unique()

In [ ]:
burnt_areas_pyrenees.head()

In [ ]:
bins = burnt_areas_pyrenees['FIREYEAR'].max() - burnt_areas_pyrenees['FIREYEAR'].min() + 1

burnt_areas_pyrenees['FIREYEAR'].hist(bins=bins)

plt.title('Number of fires per year')
plt.xlabel('Year')
plt.ylabel('Number of fires')

plt.show()

In [ ]:
burnt_areas_pyrenees['CLASS'].value_counts().plot(kind='bar')

plt.title('Class of fires')

plt.show()

In [ ]:
burnt_areas_pyrenees['COMMUNE'].value_counts()[:10].plot(kind='bar')

plt.title('Top 10 communes with the highest number of fires')

plt.show()


### Protected Areas

In [ ]:
protected_areas = gpd.read_file(protected_areas_path)
protected_areas.head()

In [ ]:
protected_areas.plot()

In [ ]:
merged = gpd.sjoin(pyrenees_boundaries, protected_areas, how="inner")

TODO : limit polygones

In [ ]:
# LAND_COVER_PATH = '../data/corine-cover/land_cover_1990/u2000_clc1990_v2020_20u1_raster100m/DATA/'
# # DEM_PATH_v1 = 'data/dem/eu_dem_v11_E30N20.TIF'
# # DEM_PATH_V0 = 'data/dem_v00/EUD_CP-DEMS_3500025000-AA.tif'
# # DEM_REPROJ_PATH = 'data/dem_v00/dem_reproj.tif'
# # DEM_REPROJ_RES_PATH = 'data/dem_v00/dem_test.tif'
# # VEGETATION_PATH = 'data/'
# BURNT_AREAS_PATH = '../data/burnt_areas/modis.ba.poly.shp'
# # PROTECTED_AREAS_PATH = 'data/ens/ens.shp'
# CONTOURS_PATH = '../data/departements-20170102-simplified/departements-20170102.shp'
# SLOPE_PATH = 'data/slope/EUD_CP-SLOP_3500025000-AA.tif'
# SLOPE_REPROJ_PATH = '../data/slope/sl_test.tif'

In [ ]:
import geopandas as gpd
import shapely.geometry
import shapely.wkt

import rasterio 
from rasterio.plot import show
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.features import rasterize

import subprocess

# CRS transformations
import pyproj

# raster operations
import os
from osgeo import gdal

import rasterio.mask 
import rasterstats
import math

### Contouring

In [ ]:
contours = gpd.read_file(CONTOURS_PATH)

In [ ]:
contours.shape

In [ ]:
contours.head()

In [ ]:
contours.plot()

We will use the department number to select the regions of interest : in this case, Pyrénées-Atlantiques (64) and Hautes-Pyrénées (65).

In [ ]:
pyrenees = contours[(contours.code_insee == '64') | (contours.code_insee == '65')]
pyrenees.plot()

An important information to note is that the CRS used is the WGS 84: as we are in 2D, the CRS is commonly named EPSG 4326.

### Slope
With the help of QGIS application, we performed the necessary transformation to simplify the CRS of the slope and DEM rasters.

In [ ]:
slope_reproj = rasterio.open(SLOPE_REPROJ_PATH)
# slope = rasterio.open(SLOPE_PATH)

In [ ]:
slope.meta

In [ ]:
slope_reproj.meta

### Digital Elevation Model

In [ ]:
%%time

# open a connection to the raster
dem = rasterio.open(DEM_PATH_V0)
dem_reproj = rasterio.open(DEM_REPROJ_PATH)
dem_test = rasterio.open(DEM_REPROJ_RES_PATH)

In [ ]:
# plot the raster
show(dem)

In [ ]:
# raster metadata
dem.meta

In [ ]:
dem_reproj.meta

In [ ]:
dem_test.meta

In [ ]:
land_cov.meta

The metadata of the raster file give us important informations : the CRS used is the ETRS_1989 or EPSG 3035 if we use the EPSG classification. The unit used is the meter, its resolution is 25x25 meters with the origin starting at (3000000.0, 3000000.0) in its corresponding CRS.  
The land cover raster had a resolution of 100x100 meters, therefore we need to aggregate the values of four non-overlapping  adjacent cells to obtain the same resolution. With the help of QGIS application, we have reprojected DEM and slope data so that it is recognized by the ESPG registry. It will help us merge the transform of the different rasters.  

In [ ]:
%%time
# we have used Qgis and gdal to perform the projections and change of resolutions 
'''
# change the resolution, transform and crs of dem data to match land cover metadata
# use rasterio.reproject() using the same crs as src and dst

with rasterio.open(DEM_PATH) as src_dem:
    print(type(src_dem))
    
    # transform for input raster
    src_transform = src_dem.transform
    # dem crs not recognized by rasterio : we assume that land cover and dem data have the same crs : epsg 3035
    dst_crs = land_cov.crs
    src_crs = land_cov.crs
    
    # calculate the transform matrix for the output
    dst_transform, width, height = calculate_default_transform(
        dst_crs,    # source CRS
        dst_crs,    # destination CRS
        src_dem.width,    # column count
        src_dem.height,  # row count
        *src_dem.bounds,  # unpacks outer boundaries (left, bottom, right, top)
    )
    
    # set properties for output
    dst_kwargs = src_dem.meta.copy()
    dst_kwargs.update(
        {
            "crs": dst_crs,
            "transform": dst_transform,
            "width": width,
            "height": height,
            "nodata": land_cov.meta['nodata'], 
        }
    )
    
    print(src_dem.meta)
    
    with rasterio.open("dem_reproj", "w", **dst_kwargs) as dst:
        reproject(
            source = src_dem,
            destination = dst,
            src_transform=src_transform,
            src_crs=src_crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest)'''

### Land Cover

In [ ]:
# 2000 land_cover data
land_cov_2000 = rasterio.open(LAND_COVER_PATH)

In [ ]:
# visualize with QGIS: memory problem
show(land_cov_2000)

In [ ]:
land_cov_2000.meta

The metadata of the raster file give us important informations : the CRS used is the ETRS_1989 or EPSG 3035 if we use the EPSG classification. The unit used is the meter, its resolution is 100x100 meters with the origin starting at (900000.0, 900000.0) in its corresponding CRS.

In [ ]:
# the memory footprint of the raster seems heavy, thus we will need to mask the data using the contours shapefile we have seen previously
layer_cov = land_cov_2000.read(1)

In [ ]:
land_cov_2000.meta

In [ ]:
dem_test.meta

Now we need to crop the raster file to the area of interest in the study that is the pyrenees dataframe we have previously defined.  
First we need to convert the CRS of the contours vector into the CRS of land cover/dem rasters.  
For simplicity, we consider that the land cover/dem have the same crs EPSG:3035 (ETRS89-LAEA)

In [ ]:
# affine transform of vector shapes ?

pyrenees_proj = pyrenees.to_crs(epsg='3035')

land_cover_mask, land_cover_transform_mask = rasterio.mask.mask(
    land_cov_2000, 
    pyrenees_proj.geometry, 
    crop=True, 
    nodata=land_cov_2000.meta['nodata']
)

dem_mask, dem_transform_mask = rasterio.mask.mask(
    dem_test, 
    pyrenees_proj.geometry,
    crop=True, 
    nodata=dem_test.meta['nodata']
)

slope_mask, slope_transform_mask = rasterio.mask.mask(
    dem_test, 
    pyrenees_proj.geometry,
    crop=True, 
    nodata=dem_test.meta['nodata']
)

# TODO: homegeneise land cover and dem data beforehand 
print('LAND COVER CROPPING')
print(land_cover_mask.squeeze().shape)
print(land_cover_transform_mask)

print('\nDEM CROPPING')
print(dem_mask.squeeze().shape)
print(dem_transform_mask)

### Burnt Areas

TODO: EDA on this file

We will use this file to label the areas of interest in this study. First we will perform an exploratory data analysis to give us insights on the nature of the data.

In [ ]:
burnt = gpd.read_file(BURNT_AREAS_PATH)

In [ ]:
burnt.shape

In [ ]:
burnt.head()

In [ ]:
burnt[burnt.COUNTRY=='FR']['id'].count()

# EDA : 
# plot total number of fires per country 
# plot total number of fires per year  

In [ ]:
burnt.groupby('COUNTRY')['id'].count().hist(bins=5)

plt.xlabel('Number of fires')
plt.ylabel('Number of countries (log)')

plt.show()

In [ ]:
burnt[(burnt.COUNTRY == 'IT') | (burnt.COUNTRY == 'PT') |
     (burnt.COUNTRY == 'ES') | (burnt.COUNTRY == 'FR')].plot()

In [ ]:
burnt['geometry'].type.value_counts()

In [ ]:
burnt['geometry'].crs

We see that the shapefile is in different Coordinate Reference System than land cover and digital elevation model data (coming from Copernicus sources). The next step is to project this shapefile from CRS EPSG 4236 to EPSG 3035.

In [ ]:
# We restrict the shapefile to fires happening in France 
burnt_pyrenees = burnt[(burnt.COUNTRY=='FR') & (burnt.PROVINCE=='Hautes-Pyrénées') | (burnt.PROVINCE == 'Pyrénées-Atlantiques')]

# change crs to espg 3035
burnt_pyrenees = burnt_pyrenees.to_crs(epsg='3035')

# we extract the year of the firedate and change the type to int
burnt_pyrenees['YEAR'] = burnt_pyrenees['FIREDATE'].apply(lambda d: int(d[:4]))
burnt_pyrenees.head()

In [ ]:
burnt[(burnt.COUNTRY=='FR') & (burnt.PROVINCE=='Hautes-Pyrénées') | (burnt.PROVINCE == 'Pyrénées-Atlantiques')].shape

In [ ]:
# rasterize the burnt areas to combine it with other rasters
# partition the fire date depending on the dates on the available land cover data

for g in burnt['geometry'].tolist():
    print((1, g))

### Protected Areas (France)

TODO: EDA with label of burnt areas

In [ ]:
protected = gpd.read_file(PROTECTED_AREAS_PATH)

In [ ]:
protected.head()

In [ ]:
protected.plot()

In [ ]:
protected['geometry'].crs

In [ ]:
dem_reproj.crs

In [ ]:
protected.to_crs(epsg='3035').plot()

### Vegetation